In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folders = glob.glob('/content/drive/MyDrive/ai_project/train/*')
all_img_list = glob.glob('/content/drive/MyDrive/ai_project/train/*/*')
root = '/content/drive/MyDrive/ai_project/train'
test_folder = '/content/drive/MyDrive/ai_project/test'

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [ ]:
res_transform = A.Compose([
                          A.Resize(300,300),
                          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                          ToTensorV2()
                          ])

In [ ]:
class BaseModel(nn.Module):#densenet
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
class BaseModel(nn.Module):#alex net
    def __init__(self, num_classes=19):
        super(BaseModel, self).__init__()
        self.backbone = models.alexnet(pretrained=True)
        self.backbone.features[0]=nn.Conv2d(3,64,kernel_size=11,stride=4,padding=2)
        self.backbone.classifier[6]=nn.Linear(4096,num_classes)

    def forward(self, x):
        x = self.backbone.features(x)
        x = x.view(-1,256*6*6)
        x = self.backbone.classifier(x)
        return x

In [ ]:
savepath = '/content/drive/MyDrive/ai_project/densenet_300.pt'
model = BaseModel(num_classes = 19)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 67.5MB/s]


In [ ]:
# model.load_state_dict(torch.load(savepath))
model = torch.jit.load(savepath)
model.to(device)
model.eval()

RecursiveScriptModule(
  original_name=BaseModel
  (backbone): RecursiveScriptModule(
    original_name=DenseNet
    (features): RecursiveScriptModule(
      original_name=Sequential
      (conv0): RecursiveScriptModule(original_name=Conv2d)
      (norm0): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu0): RecursiveScriptModule(original_name=ReLU)
      (pool0): RecursiveScriptModule(original_name=MaxPool2d)
      (denseblock1): RecursiveScriptModule(
        original_name=_DenseBlock
        (denselayer1): RecursiveScriptModule(
          original_name=_DenseLayer
          (norm1): RecursiveScriptModule(original_name=BatchNorm2d)
          (relu1): RecursiveScriptModule(original_name=ReLU)
          (conv1): RecursiveScriptModule(original_name=Conv2d)
          (norm2): RecursiveScriptModule(original_name=BatchNorm2d)
          (relu2): RecursiveScriptModule(original_name=ReLU)
          (conv2): RecursiveScriptModule(original_name=Conv2d)
        )
        (denselayer2)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
test_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+test['img_path'].values, None, res_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)

In [ ]:
def inference(model, res_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(res_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    pred_kor=label_decoder(preds)

    # preds = le.inverse_transform(preds)
    return pred_kor

In [ ]:
preds = inference(model, test_loader, device)
preds = [str(x) for x in preds]

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ai_project/sample_submission.csv',encoding='utf-8-sig')
submit['label'] = preds
submit.to_csv('/content/drive/MyDrive/ai_project/dense_300_5_22.csv', index=False, encoding='utf-8-sig')